# Extra notebook - Paper figures, multi-city analysis
## Project: Growing Urban Bicycle Networks

This notebook creates the figures for the research paper and runs further multi-city analysis. Before running this notebook, the notebooks 03, 04, and 05 need to have been run for all 6 parameter sets, for all the cities in cities_all.csv. Results on car-constricted networks, if available from X_carconstricted_metrics, can be plotted by 
setting `plotconstricted = True`.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2021-02-08  
Last modified: 2024-09-24

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"
plotnotitle = True # If True, will not plot a text title for the plots on top left
plotconstricted = False # If True, will add plots about constricted street network metrics

### Setup

In [ ]:
%run -i path.py
%run -i setup.py
plt.style.use(PATH["parameters"] + 'plotstyle.mplstyle')

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

### Constants

In [ ]:
# Run all parameter sets
poi_source_list = ["grid", "railwaystation"]
prune_measure_list = ["betweenness", "closeness", "random"]
combs = list(itertools.product(poi_source_list, prune_measure_list))
print(combs)

In [ ]:
numcitiestotal = len(cities.keys())
numcitiestotal

### Load all results

In [ ]:
analysis_result = {}
for p in poi_source_list:
    analysis_result[p] = {}
    for m in prune_measure_list:
        analysis_result[p][m] = {}
        
        for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
            filename = placeid + '_poi_' + p + "_" + m + ".csv"
            analysis_result[p][m][placeid] = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
            if len(analysis_result[p][m][placeid]) == 0:
                analysis_result[p][m][placeid] = analysis_result[p][m][list(cities.keys())[0]]
                for n in analysis_result[p][m][placeid].dtype.names:
                    analysis_result[p][m][placeid][n] = [-1]*len(analysis_result[p][m][placeid][n])

In [ ]:
analysis_existing = {}
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    filename = placeid + '_existing.csv'
    analysis_existing[placeid] = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True, usecols = (1,2,3,4,5,6,7,8,9,10,11))

In [ ]:
analysis_constricted = {}
if plotconstricted:
    for p in poi_source_list:
        analysis_constricted[p] = {}
        for m in prune_measure_list:
            analysis_constricted[p][m] = {}
            
            for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
                f = PATH["results_constricted"] + "results_" + p + "_" + m + "/metrics_" + p + "_" + m + "/" + placeid + "_carconstrictedbike_poi_" + p + "_" + m + ".csv"
                if os.path.isfile(f):
                    temp = np.loadtxt(f, delimiter=',', usecols = (2,3,4,5,6,7,8,9,10), skiprows=2)
                    if np.shape(temp)[0] != 2: # we dont consider large cities for the avg where we only calculated 2 values
                        analysis_constricted[p][m][placeid] = temp

## Directness and Efficiency for all cities

In this section, we want to check the hypothesis that there is a "dip" in the metrics of directness and efficiency, i.e. that there is a U-shaped form where the metric starts at a high value, then falls  (due to percolation / emergence of the giant component) and then grows back to an intermediate value. We do that for each combination of [('grid', 'betweenness'), ('grid', 'closeness'), ('railwaystation', 'betweenness'), ('railwaystation', 'closeness')]

To do that, first we select all cities where the minimum values of the metric are to the right of the maximum value. For those cities we then only plot the (x,y) pairs of the min, max, and end values.

### Directness min/max/end

In [ ]:
directness_lcc = {}
for p in poi_source_list:
    directness_lcc[p] = {}
    for m in prune_measure_list:
        directness_lcc[p][m] = {}
        directness_lcc[p][m]["x"] = {}
        directness_lcc[p][m]["y"] = {}

        directness_lcc[p][m]["y"]["min"] = [min(analysis_result[p][m][placeid]["directness_lcc"]) for placeid in cities.keys()]
        directness_lcc[p][m]["y"]["max"] = [max(analysis_result[p][m][placeid]["directness_lcc"]) for placeid in cities.keys()]
        directness_lcc[p][m]["y"]["end"] = [analysis_result[p][m][placeid]["directness_lcc"][-1] for placeid in cities.keys()]

        directness_lcc[p][m]["x"]["min"] = [np.where(analysis_result[p][m][placeid]["directness_lcc"] == min(analysis_result[p][m][placeid]["directness_lcc"]))[0][-1] for placeid in cities.keys()]
        directness_lcc[p][m]["x"]["max"] = [np.where(analysis_result[p][m][placeid]["directness_lcc"] == max(analysis_result[p][m][placeid]["directness_lcc"]))[0][-1] for placeid in cities.keys()]
        directness_lcc[p][m]["x"]["end"] = [np.where(analysis_result[p][m][placeid]["directness_lcc"] == analysis_result[p][m][placeid]["directness_lcc"][-1])[0][-1] for placeid in cities.keys()]       

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (10, 8), squeeze = True)
axes = axes.flatten()

# Plot data
for i,ax in enumerate(axes):
    ind = np.where(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["min"]) > np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["max"]))[0]
    print(str(len(ind)) + "/" + str(numcitiestotal) + " cities found with x_min>x_max for "+ str(combs[i]))
    ax.plot([np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["max"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["min"])[ind]], [np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["max"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["min"])[ind]], ':', color="red", alpha=0.3);
    
#     ind = np.where(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["min"]) >= 0)[0]
    ax.plot([np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["min"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["end"])[ind]], [np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["min"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["end"])[ind]], ':', color="green", alpha=0.3);
    
    ax.plot(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["max"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["max"])[ind], '^r', label='max');
    ax.plot(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["min"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["min"])[ind], 'vg', label='min');
    ax.plot(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["end"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["end"])[ind], 'ok', label='end', markerfacecolor='none');
    
    ax.set_xlabel(combs[i][1] + ' quantile')
    ax.set_ylabel('Directness of LCC')
    ax.set_xlim([-1,40])
    ax.set_ylim([0.4,1]) #0.35
    if i == 0: ax.legend(loc='lower right');
    if i == 1 or i == 3:
        ax.set_ylabel('')
        ax.set_yticklabels([])
        ax.text(42, 0.8, combs[i][0], rotation = 90, horizontalalignment = "center", verticalalignment='center')
    if i == 0 or i == 1:
        ax.set_xlabel('')
        ax.set_xticklabels([])

### Efficiency min/max/end

In [ ]:
efficiency_global = {}
for p in poi_source_list:
    efficiency_global[p] = {}
    for m in prune_measure_list:
        efficiency_global[p][m] = {}
        efficiency_global[p][m]["x"] = {}
        efficiency_global[p][m]["y"] = {}

        efficiency_global[p][m]["y"]["min"] = [min(analysis_result[p][m][placeid]["efficiency_global"]) for placeid in cities.keys()]
        efficiency_global[p][m]["y"]["max"] = [max(analysis_result[p][m][placeid]["efficiency_global"]) for placeid in cities.keys()]
        efficiency_global[p][m]["y"]["end"] = [analysis_result[p][m][placeid]["efficiency_global"][-1] for placeid in cities.keys()]

        efficiency_global[p][m]["x"]["min"] = [np.where(analysis_result[p][m][placeid]["efficiency_global"] == min(analysis_result[p][m][placeid]["efficiency_global"]))[0][-1] for placeid in cities.keys()]
        efficiency_global[p][m]["x"]["max"] = [np.where(analysis_result[p][m][placeid]["efficiency_global"] == max(analysis_result[p][m][placeid]["efficiency_global"]))[0][-1] for placeid in cities.keys()]
        efficiency_global[p][m]["x"]["end"] = [np.where(analysis_result[p][m][placeid]["efficiency_global"] == analysis_result[p][m][placeid]["efficiency_global"][-1])[0][-1] for placeid in cities.keys()]       

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (10, 8), squeeze = True)
axes = axes.flatten()

# Plot data
for i,ax in enumerate(axes):
   
    ind = np.where(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"]) > np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["max"]))[0]
    print(str(len(ind)) + "/" + str(numcitiestotal) + " cities found with x_min>x_max for "+ str(combs[i]))
    ax.plot([np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["max"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"])[ind]], [np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["max"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["min"])[ind]], ':', color="red", alpha=0.3);
    
#     ind = np.where(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["min"]) >= 0)[0]
    ax.plot([np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["end"])[ind]], [np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["min"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["end"])[ind]], ':', color="green", alpha=0.3);
    
    ax.plot(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["max"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["max"])[ind], '^r', label='max');
    ax.plot(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["min"])[ind], 'vg', label='min');
    ax.plot(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["end"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["end"])[ind], 'ok', label='end', markerfacecolor='none');
    
    ax.set_xlabel(combs[i][1] + ' quantile')
    ax.set_ylabel('Global Efficiency')
    ax.set_xlim([-1,40])
    ax.set_ylim([0,1])
    if i == 0: ax.legend(loc='lower center');
    if i == 1 or i == 3:
        ax.set_ylabel('')
        ax.set_yticklabels([])
        ax.text(42, 0.55, combs[i][0], rotation = 90, horizontalalignment = "center", verticalalignment='center')
    if i == 0 or i == 1:
        ax.set_xlabel('')
        ax.set_xticklabels([])

### Plotting full curves

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 4, figsize=(1200/plotparam["dpi"], 300/plotparam["dpi"]), dpi=plotparam["dpi"], squeeze = True)
axes = axes.flatten()

for i, comb in enumerate(combs):
    p = comb[0]
    m = comb[1]
    ind = np.where(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"]) > np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["max"]))[0]
    print(str(len(ind)) + "/" + str(numcitiestotal) + " cities found with x_min>x_max for "+ str(combs[i]))
    print([list(cities.keys())[i] for i in ind])
    
    ax = axes[i]

    for i in ind:
        placeid = list(cities.keys())[i]
        tmp, = ax.plot(prune_quantiles, analysis_result[p][m][placeid]["efficiency_global_routed"])
        tmp.set_label('_hidden')
    ax.set_title(comb)

Next: Plotting single cities with the biggest dips for only grid,betweenness and railwaystation, betweenness

In [ ]:
[combs[0],combs[2]]

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 15, figsize=(1800/plotparam["dpi"], 300/plotparam["dpi"]), dpi=plotparam["dpi"], squeeze = True)
axes = axes.flatten()

for i, comb in enumerate([combs[0],combs[2]]):
    p = comb[0]
    m = comb[1]
    ind = np.where(np.asarray(efficiency_global[combs[i*2][0]][combs[i*2][1]]["x"]["min"]) > np.asarray(efficiency_global[combs[i*2][0]][combs[i*2][1]]["x"]["max"]))[0]
    print(str(len(ind)) + "/" + str(numcitiestotal) + " cities found with x_min>x_max for "+ str(combs[i]))
    print([list(cities.keys())[i] for i in ind])
    
    
    for j in range(len(ind)):
        ax = axes[j + i*len(ind)]
        placeid = list(cities.keys())[ind[j]]
        tmp, = ax.plot(prune_quantiles, analysis_result[p][m][placeid]["efficiency_global_routed"])
        tmp.set_label('_hidden')
        ax.set_title(placeid)
        ax.set_ylim([0,1])

Now plot only the biggest dips

In [ ]:
cities.keys()

In [ ]:
bigdips = ["delft", "copenhagen", "boston", "paris", "barcelona", "sheffield"]

fig, axes = plt.subplots(nrows = 2, ncols = 3, figsize=(300/plotparam["dpi"], 220/plotparam["dpi"]), dpi=plotparam["dpi"], squeeze = True)
axes = axes.flatten()

i = 0
comb = combs[0]
p = comb[0]
m = comb[1]

for j,placeid in enumerate(bigdips):  
    ax = axes[j]
    tmp, = ax.plot(prune_quantiles, analysis_result[p][m][placeid]["efficiency_global_routed"], **plotparam_analysis["bikegrown_" + m])
    ax.text(0.96, 0.08, cities[placeid]["name"], fontsize=8, horizontalalignment='right')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xticks([0, 0.5, 1])
    if j==4:
        ax.set_xlabel('Betweenness quantile')
    if j <= 2:
        ax.set_xlabel('')
        ax.set_xticklabels([])
    else:
        ax.set_xticklabels([0,0.5,1])
    ax.set_yticks([0,0.25,0.5,0.75,1])
    if j % 3 == 0:
        ax.set_yticklabels([0,0.25,0.5,0.75,1])
    else:
        ax.set_ylabel('')
        ax.set_yticklabels([])
    if j == 1:
        ax.set_title('Dips in global efficiency')
plt.subplots_adjust(top = 0.90, bottom = 0.16, left = 0.12, right = 0.97, wspace = 0.22, hspace = 0.22)
fig.savefig(PATH["plots"] + "/" + 'dipsglobalefficiency_poi_' + p + '.eps', facecolor = "white", edgecolor = 'none')

Now figure for paper: Directness and Global eff, for Boston (big left) and 4 cities (small right)

In [ ]:
from matplotlib.font_manager import findfont, FontProperties
font = findfont(FontProperties(family=['sans-serif']))
font

In [ ]:
smallmultiples = ["montreal", "mumbai", "paris", "tokyo"]
checkpoints = [0.025, 0.1, 0.2]
checkpoints_x = [0,3,7]
xticks = [0, 0.1, 0.2,0.3,0.4]
yticks_top = [0.6,0.7,0.8,0.9]
yticks_bot = [0,0.2,0.4,0.6,0.8,1]
ytextpos_top = [0.87, 0.835]
ytextpos_bot = [0.04, 0.08]

fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize=(300/plotparam["dpi"], 300/plotparam["dpi"]), dpi=plotparam["dpi"], squeeze = True)

i = 0
comb = combs[0]
p = comb[0]
m = comb[1]
smallmultipleindex = 0

for row in [1,2,3,4]:
    if row < 3:
        yticks = yticks_top
        ytextpos = ytextpos_top
    else:
        yticks = yticks_bot
        ytextpos = ytextpos_bot
    for col in [1,2,3,4]:
        i += 1
        placeid = "boston"
        if col >= 3: # Right half: small multiple
            ax = plt.subplot(4,4,i)
            placeid = smallmultiples[smallmultipleindex]
            smallmultipleindex += 1
            smallmultipleindex = smallmultipleindex % len(smallmultiples)
            ax.text(0.96*xticks[-1], ytextpos[1], cities[placeid]["name"], fontsize=8, horizontalalignment='right')
            if row <= 2:
                ax.plot(prune_quantiles, analysis_result[p][m][placeid]["directness_all_linkwise"], **plotparam_analysis["bikegrown_" + m])
            else:
                ax.plot(prune_quantiles, analysis_result[p][m][placeid]["efficiency_global_routed"], **plotparam_analysis["bikegrown_" + m])
            ax.set_yticklabels([])
            ax.set_yticks(yticks)
            ax.set_xticks(xticks)
            if row == 4:
                ax.set_xticklabels('')
            else:
                ax.set_xticklabels('')
            
        elif (row == 1 and col == 1): # Top left
            ax = plt.subplot(4,4,(1,6))
            ax.plot(prune_quantiles, analysis_result[p][m][placeid]["directness_all_linkwise"], **plotparam_analysis["bikegrown_" + m])
            ax.text(0.98*xticks[-1], ytextpos[0], cities[placeid]["name"], fontsize=8, horizontalalignment='right')
            ax.set_yticks(yticks)
            ax.set_yticklabels(yticks)
            ax.set_ylabel('Directness')
            ax.set_xticks(xticks)
            ax.set_xticklabels([])
            for j,c in enumerate(checkpoints):
                ax.plot([c, c], [0, 1], ":k", linewidth = 1)
                ax.plot([c], [analysis_result[p][m][placeid]["directness_all_linkwise"][checkpoints_x[j]]], "ok", fillstyle='none', markersize = 5)
            
        elif (row == 3 and col == 1): # Bottom left
            ax = plt.subplot(4,4,(9,14))
            ax.text(0.98*xticks[-1], ytextpos[0], cities[placeid]["name"], fontsize=8, horizontalalignment='right')
            ax.plot(prune_quantiles, analysis_result[p][m][placeid]["efficiency_global_routed"], **plotparam_analysis["bikegrown_" + m])
            ax.set_yticks(yticks)
            ax.set_yticklabels(yticks)
            ax.set_ylabel('Global efficiency')
            ax.set_xlabel('Betweenness quantile $q_B$')
            ax.set_xticks(xticks)
            ax.set_xticklabels(xticks)
            for j,c in enumerate(checkpoints):
                ax.plot([c, c], [0, 1], ":k", linewidth = 1)
                ax.plot([c], [analysis_result[p][m][placeid]["efficiency_global_routed"][checkpoints_x[j]]], "ok", fillstyle='none', markersize = 5)
            
        ax.set_xlim([0, 0.4])
        if row < 3:
            ax.set_ylim(0.6, 0.9)
        else:
            ax.set_ylim(0, 1)
        
plt.subplots_adjust(top = 0.98, bottom = 0.13, left = 0.14, right = 0.97, wspace = 0.22, hspace = 0.22)
            
            

fig.savefig(PATH["plots"] + "/" + 'smallmultiples_poi_' + p + '.eps', facecolor = "white", edgecolor = 'none')

## Comparing grown with existing metrics

Here we plot for some metrics how much higher the metric of the grown network is compared to the existing network of same length. This only works for cities that have a small enough existing length which is at some point reached by the grown network.

In [ ]:
comp_keys = ["length_lcc", "efficiency_global_routed", "efficiency_local_routed", "coverage"]
comp_labels = ["Length of LCC", "Global Efficiency", "Local Efficiency", "Coverage"]

cities_here = []

for p in poi_source_list:
    for m in prune_measure_list:
        numcities = 0
        values = []

        if debug: fig = plt.figure(figsize=(400/plotparam["dpi"], 400/plotparam["dpi"]), dpi=plotparam["dpi"])
        if debug: plt.semilogy([min(x), max(x)], [1,1], "k--")
        for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
            length_existing = analysis_existing[placeid][analysis_existing_rowkeys["biketrack"]]["length"]
            if length_existing and np.argmax(analysis_result[p][m][placeid]["length"] > length_existing):
                numcities += 1
                id_samelen = np.argmax(analysis_result[p][m][placeid]["length"] > length_existing) - 1

                x = list(range(len(comp_keys)))
                y = [[analysis_result[p][m][placeid][comp_keys[i]][id_samelen] / analysis_existing[placeid][analysis_existing_rowkeys["biketrack"]][comp_keys[i]]] for i in x]
                values.append(y)
                cities_here.append(placeid)
                if debug: plt.semilogy(x, y, "o")
                print(id_samelen)

        print(str(numcities) + " cities found where L_grown=L for " + p + " | " + m)
        print(cities_here)
        values = np.log10(np.array(values))
        fig = plt.figure(figsize=(200/plotparam["dpi"], 100/plotparam["dpi"]), dpi=plotparam["dpi"])
        axes = fig.add_axes([0, 0, 1, 1])
        axes.plot([0,0], [min(x), max(x)+2],"k:", linewidth=0.5)
        # https://stackoverflow.com/questions/18500011/horizontal-box-plots-in-matplotlib-pandas#56088231
        axes.boxplot([values[np.isfinite(values[:,i]).flatten(),i].flatten().tolist() for i in x], vert=False, showfliers=False); #with isfinite we exclude zeros, infs, or nans which sometimes happen for e.g. local efficiency
        axes.set_xlabel('$M_{syn}/M_{real}$ | $L_{syn}=L_{real}$')
        axes.set_title('Synthetic versus real networks')
        axes.set_yticklabels([comp_labels[i] for i in x]);
        axes.set_ylim([min(x)+0.5, max(x)+1.5])
        # fig.autofmt_xdate(rotation=45)

        axes.set_xlim([-0.7, 1.55])
#         axes.set_xlim([-0.35, 1.55])
        axes.set_xticks([-0.301,0,0.301, 0.699, 1, 1.301])
        axes.set_xticklabels([0.5, 1, 2,5, 10, 20])
        axes.xaxis.set_minor_locator(matplotlib.ticker.FixedLocator(np.log10(np.concatenate((np.linspace(0,1,10, endpoint=False),np.linspace(1, 10,9, endpoint=False),np.linspace(10, 100,9, endpoint=False))))))
        axes.text(1.5, 4, p + " | " + m, fontsize=8, horizontalalignment='right');
        fig.savefig(PATH["plots"] + 'grownvsexisting_poi_' + p + "_" + m + '.eps', facecolor = "white", edgecolor = 'none', bbox_inches="tight")

## Existing cover plot

We do this for Milan.

In [ ]:
# Real Milan

# %%capture
    
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Plotting network covers")

    # Load networks
    G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
    G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')
    map_center = nxdraw(G_carall, "carall")
    
    
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
        res = pickle.load(f)
    
    # Load covers
    filename = placeid + "_"  + "existing_covers"
    with open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb') as f:
        cov_car = pickle.load(f)['carall']
    with open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb') as f:
        cov_biketrack_onstreet = pickle.load(f)['biketrack_onstreet']
    with open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb') as f:
        cov_bikeable_offstreet = pickle.load(f)['bikeable_offstreet']
    # Merge on and offstreet
    cov_bike = ops.unary_union([cov_biketrack_onstreet, cov_bikeable_offstreet])
    
    # Construct and plot patches from covers
    patchlist_car, patchlist_car_holes = cov_to_patchlist(cov_car, map_center)
    fig = initplot()

    # Covers
    axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
    patchlist_bike, patchlist_bike_holes = cov_to_patchlist(cov_bike, map_center)

    # We have this contrived order due to alphas, holes, and matplotlib's inability to draw polygon patches with holes. This only works because the car network is a superset of the bike network.
    # car orange, bike white, bike blue, bike holes white, bike holes orange, car holes white
    patchlist_combined = patchlist_car + patchlist_bike + patchlist_bike + patchlist_bike_holes+ patchlist_bike_holes + patchlist_car_holes
    pc = PatchCollection(patchlist_combined)
    colors = np.array([[255/255,115/255,56/255,0.2] for _ in range(len(patchlist_car))]) # car orange
    if len(patchlist_bike):
        colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_bike))], axis = 0) # bike white
        colors = np.append(colors, [[86/255,220/255,244/255,0.4] for _ in range(len(patchlist_bike))], axis = 0) # bike blue
    if len(patchlist_bike_holes):
        colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_bike_holes))], axis = 0) # bike holes white
    if len(patchlist_bike_holes):
        colors = np.append(colors, [[255/255,115/255,56/255,0.2] for _ in range(len(patchlist_bike_holes))], axis = 0) # bike holes orange
    if len(patchlist_car_holes):
        colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_car_holes))], axis = 0) # car holes white
    pc.set_facecolors(colors)
    pc.set_edgecolors(np.array([[0,0,0,0.4] for _ in range(len(patchlist_combined))])) # remove this line if the outline of the full coverage should remain
    axes.add_collection(pc)
    axes.set_aspect('equal')
    axes.set_xmargin(0.01)
    axes.set_ymargin(0.01)
    axes.plot()

    # Networks
    nxdraw(G_carall, "carall", map_center)
    nxdraw(G_biketrack, "bikegrown", map_center, nodesize = nodesize_grown)
    nxdraw(G_biketrack.clusters().giant(), "highlight_biketrack", map_center, nodesize = nodesize_grown)
    plt.savefig(PATH["plots_networks"] + placeid + "/" + placeid + '_bikecarcover.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()

In [ ]:
# Synthetic Milan

# %%capture
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Plotting network covers")

    # Load networks
    G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
    G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')
    map_center = nxdraw(G_carall, "carall")
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    nodesize_poi = nodesize_from_pois(nnids)
    
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
        res = pickle.load(f)
    
    # Load covers
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + "_covers"
    with open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb') as f:
        covs = pickle.load(f)
    filename = placeid + "_"  + "existing_covers"
    with open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb') as f:
        cov_car = pickle.load(f)['carall']
    
    # Construct and plot patches from covers
    patchlist_car, patchlist_car_holes = cov_to_patchlist(cov_car, map_center)
    GT = res["GTs"][16]
    prune_quantile = res["prune_quantiles"][16]
    cov = list(covs.values())[16]
    fig = initplot()

    # Covers
    axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
    patchlist_bike, patchlist_bike_holes = cov_to_patchlist(cov, map_center)

    # We have this contrived order due to alphas, holes, and matplotlib's inability to draw polygon patches with holes. This only works because the car network is a superset of the bike network.
    # car orange, bike white, bike blue, bike holes white, bike holes orange, car holes white
    patchlist_combined = patchlist_car + patchlist_bike + patchlist_bike + patchlist_bike_holes+ patchlist_bike_holes + patchlist_car_holes
    pc = PatchCollection(patchlist_combined)
    colors = np.array([[255/255,115/255,56/255,0.2] for _ in range(len(patchlist_car))]) # car orange
    if len(patchlist_bike):
        colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_bike))], axis = 0) # bike white
        colors = np.append(colors, [[86/255,220/255,244/255,0.4] for _ in range(len(patchlist_bike))], axis = 0) # bike blue
    if len(patchlist_bike_holes):
        colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_bike_holes))], axis = 0) # bike holes white
    if len(patchlist_bike_holes):
        colors = np.append(colors, [[255/255,115/255,56/255,0.2] for _ in range(len(patchlist_bike_holes))], axis = 0) # bike holes orange
    if len(patchlist_car_holes):
        colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_car_holes))], axis = 0) # car holes white
    pc.set_facecolors(colors)
    pc.set_edgecolors(np.array([[0,0,0,0.4] for _ in range(len(patchlist_combined))])) # remove this line if the outline of the full coverage should remain
    axes.add_collection(pc)
    axes.set_aspect('equal')
    axes.set_xmargin(0.01)
    axes.set_ymargin(0.01)
    axes.plot()

    # Networks
    nxdraw(G_carall, "carall", map_center)
    nxdraw(GT, "highlight_biketrack", map_center, nodesize = nodesize_grown)
    plt.savefig(PATH["plots_networks"] + placeid + "/" + placeid + '_bikesynthcarcover.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()

## Average analysis plot

Here we plot one analysis figure as an average over all cities.

### Each parameter set as a single figure

In [ ]:
# %%capture

for poi_source, prune_measure in combs:
    print(poi_source, prune_measure)
        
    analysis_result_city = {}
    numcities = 0
    for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):

        # PLOT Analysis
        filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".csv"
        analysis_result_city_temp = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
        if len(analysis_result_city_temp) == 0: # Discard if no results (for example no railwaystations)
            print(placeid + ": No analysis results available")
            continue
        else:
            numcities += 1
            analysis_result_city[placeid] = analysis_result_city_temp
            metric_keys = analysis_result_city[placeid].dtype.names
    
    # All cities are loaded, now create the average values
    analysis_result_AVG = {}
    for metric in metric_keys:
        temp = np.zeros([40, numcities])
        for i, placeid_this in enumerate(analysis_result_city.keys()):
            temp[:, i] = analysis_result_city[placeid_this][metric]
        analysis_result_AVG[metric] = np.mean(temp, axis = 1)
            
    
                    
    # Plot
    nc = 5
    fig, axes = plt.subplots(nrows = 2, ncols = nc, figsize = (16, 6))
    keys_metrics = {"length": "Length [km]","coverage": "Coverage [km$^2$]","overlap_biketrack": "Overlap Protected","directness": "Directness","efficiency_global": "Global Efficiency",
            "length_lcc": "Length of LCC [km]","poi_coverage": "Seed points covered","overlap_bikeable": "Overlap Bikeable","components": "Components","efficiency_local": "Local Efficiency"}

    for i, ax in enumerate(axes[0]):
        key = list(keys_metrics.keys())[i]
        if key in ["overlap_biketrack", "overlap_bikeable"]:
            ax.plot(prune_quantiles, analysis_result_AVG[key] / analysis_result_AVG["length"], **plotparam_analysis["bikegrown"])
        elif key in ["efficiency_global", "efficiency_local"]:
            ax.plot(prune_quantiles, analysis_result_AVG[key], **plotparam_analysis["bikegrown_abstract"])
            tmp, = ax.plot(prune_quantiles, analysis_result_AVG[key+"_routed"], **plotparam_analysis["bikegrown"])
            tmp.set_label('_hidden')
        elif key in ["length", "length_lcc"]: # Convert m->km
            ax.plot(prune_quantiles, analysis_result_AVG[key]/1000, **plotparam_analysis["bikegrown"])
        else:
            ax.plot(prune_quantiles, analysis_result_AVG[key], **plotparam_analysis["bikegrown"])

        if i == 0:
            ymax0 = ax.get_ylim()[1]
            ax.set_ylim(0, ymax0)
            ax.text(-0.15, ymax0*1.25, "Average city" + " (" + poi_source + " | " + prune_measure + ")", fontsize=16, horizontalalignment='left')
            ax.legend(loc = 'upper left')
        if i == 4:
            ax.legend(loc = 'best')


        set_analysissubplot(key)
        ax.set_title(list(keys_metrics.values())[i])
        ax.set_xlabel('')
        ax.set_xticklabels([])


    for i, ax in enumerate(axes[1]):
        key = list(keys_metrics.keys())[i+nc]
        if key in ["overlap_biketrack", "overlap_bikeable"]:
            ax.plot(prune_quantiles, analysis_result_AVG[key] / analysis_result_AVG["length"], **plotparam_analysis["bikegrown"])
        elif key in ["efficiency_global", "efficiency_local"]:
            ax.plot(prune_quantiles, analysis_result_AVG[key], **plotparam_analysis["bikegrown_abstract"])
            ax.plot(prune_quantiles, analysis_result_AVG[key+"_routed"], **plotparam_analysis["bikegrown"])
        elif key in ["length", "length_lcc"]: # Convert m->km
            ax.plot(prune_quantiles, analysis_result_AVG[key]/1000, **plotparam_analysis["bikegrown"])
        else:
            ax.plot(prune_quantiles, analysis_result_AVG[key], **plotparam_analysis["bikegrown"])

        if i == 0:
            ax.set_ylim(0, ymax0)
        set_analysissubplot(key)
        ax.set_title(list(keys_metrics.values())[i+nc])
        ax.set_xlabel(prune_measure + ' quantile')
        if key in ["poi_coverage"]:
            # https://stackoverflow.com/questions/30914462/matplotlib-how-to-force-integer-tick-labels
            ax.yaxis.set_major_locator(MaxNLocator(integer=True)) 

    plt.subplots_adjust(top = 0.87, bottom = 0.09, left = 0.05, right = 0.97, wspace = 0.25, hspace = 0.25)
    if plotconstricted:
        fig.savefig(PATH["plots"] + "/" + 'averagecity_analysis_poi_' + poi_source + "_" + prune_measure + '.png', facecolor = "white", edgecolor = 'none')
    else:
        fig.savefig(PATH["plots"] + "/" + 'averagecity_analysis_poi_' + poi_source + "_" + prune_measure + '_noconstr.png', facecolor = "white", edgecolor = 'none')
    plt.close()

### Figures with bundled parameter sets

In [ ]:
def set_analysissubplot_special(key, p):
    if key in ["overlap_bikeable"]:
        ax.set_ylim(0.08,0.22)
    if key in ["overlap_biketrack"]:
        ax.set_ylim(0.015,0.065)
    
    if p == "grid":
        if key in ["components"]:
            ax.set_ylim(top = 25)
        if key in ["efficiency_local"]:
            ax.set_ylim(top = 0.25)
#         if key in ["overlap_bikeable"]:
#             ax.set_ylim(top = 0.25)
#         if key in ["overlap_biketrack"]:
#             ax.set_ylim(top = 0.06)
        if key in ["directness"]:
            ax.set_ylim(bottom = 0.43)
            ax.set_ylim(top = 0.81)
        if key in ["directness_all_linkwise"]:
            ax.set_ylim(bottom = 0.47)
            ax.set_ylim(top = 0.84)
    elif p == "railwaystation":
        if key in ["components"]:
            ax.set_ylim(top = 25)
        if key in ["efficiency_local"]:
            ax.set_ylim(top = 0.25)
#         if key in ["overlap_bikeable"]:
#             ax.set_ylim(top = 0.25)
#         if key in ["overlap_biketrack"]:
#             ax.set_ylim(top = 0.06)

In [ ]:
# %%capture
plotparam_analysis["constricted"] = {"linewidth": 1.5, "color": '#D22A0E', "linestyle": "solid", "label": "Betw. cars"}
plotparam_analysis["mst"] = {"linewidth": 1, "color": '#aaaaaa', "linestyle": "dotted", "label": "MST"}
plotparam_analysis["bikegrown_betweenness"] = {"linewidth": 2.5, "color": '#0eb6d2', "linestyle": "solid", "label": "Betw."}
plt.style.use(PATH["parameters"] + 'plotstyle.mplstyle')
# Run all parameter sets
prune_measure_list = ["betweenness", "closeness", "random"]
poi_source = "grid" # railwaystation grid

analysis_result_city = {}
analysis_result_AVG = {}
analysis_constricted_AVG = {}
analysis_mst_result_city = {}
for m in prune_measure_list:
    analysis_result_city[m] = {}
    numcities = 0
    for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):

        # PLOT Analysis
        filename = placeid + '_poi_' + poi_source + "_mst.csv"
        analysis_mst_result_city_temp = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
        
        filename = placeid + '_poi_' + poi_source + "_" + m + ".csv"
        analysis_result_city_temp = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
        if len(analysis_result_city_temp) == 0: # Discard if no results (for example no railwaystations)
            print(placeid + ": No analysis results available")
            continue
        else:
            numcities += 1
            analysis_result_city[m][placeid] = analysis_result_city_temp
            metric_keys = analysis_result_city[m][placeid].dtype.names
            analysis_mst_result_city[placeid] = analysis_mst_result_city_temp

    # All cities are loaded, now create the average values
    analysis_result_AVG[m] = {}
    for metric in metric_keys:
        temp = np.zeros([40, numcities])
        for i, placeid_this in enumerate(analysis_result_city[m].keys()):
            temp[:, i] = analysis_result_city[m][placeid_this][metric]
        analysis_result_AVG[m][metric] = np.mean(temp, axis = 1)
        
    analysis_mst_result_AVG = {}
    for metric in metric_keys:
        temp = np.zeros([1, numcities])
        for i, placeid_this in enumerate(analysis_mst_result_city.keys()):
            temp[:, i] = analysis_mst_result_city[placeid_this][metric]
        analysis_mst_result_AVG[metric] = np.mean(temp, axis = 1)
    print(analysis_mst_result_AVG)

    analysis_constricted_AVG[m] = {}
    for metric in list(range(9)): # 9 constricted fields are loaded
        temp = np.zeros([40, len(analysis_constricted[poi_source][m].keys())]) 
        for i, placeid_this in enumerate(analysis_constricted[poi_source][m].keys()):
            temp[:, i] = analysis_constricted[poi_source][m][placeid_this][:, metric]
        analysis_constricted_AVG[m][metric] = np.mean(temp, axis = 1)

# Plot
nc = 4
fig, axes = plt.subplots(nrows = 2, ncols = nc, figsize=(640/plotparam["dpi"], 320/plotparam["dpi"]), dpi=plotparam["dpi"])
keys_metrics = {"length": "Length [km]","coverage": "Coverage [km$^2$]", "directness_all_linkwise": "Directness", "efficiency_global": "Global Efficiency",
        "length_lcc": "Length of LCC [km]","poi_coverage": "Seed points covered","components": "Components","efficiency_local": "Local Efficiency"}

for i, ax in enumerate(axes[0]):
    key = list(keys_metrics.keys())[i]
    for m in prune_measure_list:
        if key in ["overlap_biketrack", "overlap_bikeable"]:
            ax.plot(prune_quantiles, analysis_result_AVG[m][key] / analysis_result_AVG[m]["length"], **plotparam_analysis["bikegrown_" + m])
        elif key in ["efficiency_global", "efficiency_local"]:
            if m == "betweenness":
                tmp, = ax.plot([0,1], [analysis_mst_result_AVG[key], analysis_mst_result_AVG[key]], **plotparam_analysis["mst"])
            ax.plot(prune_quantiles, analysis_result_AVG[m][key+"_routed"], **plotparam_analysis["bikegrown_" + m])
        elif key in ["length", "length_lcc"]: # Convert m->km
            if m == "betweenness":
                ax.plot([0,1], [analysis_mst_result_AVG[key]/1000, analysis_mst_result_AVG[key]/1000], **plotparam_analysis["mst"])
            ax.plot(prune_quantiles, analysis_result_AVG[m][key]/1000, **plotparam_analysis["bikegrown_" + m])
        else:
            if m == "betweenness":
                ax.plot([0,1], [analysis_mst_result_AVG[key], analysis_mst_result_AVG[key]], **plotparam_analysis["mst"])
            ax.plot(prune_quantiles, analysis_result_AVG[m][key], **plotparam_analysis["bikegrown_" + m])
            
        if key == "directness_all_linkwise" and plotconstricted and m == "betweenness": # Only show betweenness - the others are similar
            tmp, = ax.plot(prune_quantiles, analysis_constricted_AVG[m][8], **plotparam_analysis["constricted"])
            tmp.set_label('_hidden')
        if key == "efficiency_global" and plotconstricted and m == "betweenness": # Only show betweenness - the others are similar
            ax.plot(prune_quantiles, analysis_constricted_AVG[m][0], **plotparam_analysis["constricted"])
                
        if key in ["directness_all_linkwise"]:
            ax.arrow(0.33, 0.62, 0, -0.04, color = "orange", width = 0.01, head_width = 0.05, head_length = 0.02)

        if i == 0:
            ymax0 = ax.get_ylim()[1]
            ax.set_ylim(0, ymax0)
            if not plotnotitle:
                ax.text(-0.15, ymax0*1.25, "Average city" + " (" + poi_source + ")", fontsize=16, horizontalalignment='left')
                
        if i == 3:
            leg = ax.legend(loc = 'best')
            leg.get_frame().set_linewidth(0.5)
            ax.set_zorder(100)
            # https://stackoverflow.com/questions/23238041/move-and-resize-legends-box-in-matplotlib
            bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)
            # Change to location of the legend. 
            xOffset = -1.26
            bb.x0 += xOffset
            bb.x1 += xOffset
            yOffset = -1.26
            bb.y0 += yOffset
            bb.y1 += yOffset
            leg.set_bbox_to_anchor(bb, transform = ax.transAxes)
            
            
        

    set_analysissubplot(key)
    set_analysissubplot_special(key, poi_source)
    ax.set_title(list(keys_metrics.values())[i])
    ax.set_xlabel('')
    ax.set_xticklabels([])


for i, ax in enumerate(axes[1]):
    key = list(keys_metrics.keys())[i+nc]
    for m in prune_measure_list:
        if key in ["overlap_biketrack", "overlap_bikeable"]:
            ax.plot(prune_quantiles, analysis_result_AVG[m][key] / analysis_result_AVG[m]["length"], **plotparam_analysis["bikegrown_" + m])
        elif key in ["efficiency_global", "efficiency_local"]:
            if m == "betweenness":
                tmp, = ax.plot([0,1], [analysis_mst_result_AVG[key], analysis_mst_result_AVG[key]], **plotparam_analysis["mst"])
            ax.plot(prune_quantiles, analysis_result_AVG[m][key+"_routed"], **plotparam_analysis["bikegrown_" + m])
        elif key in ["length", "length_lcc"]: # Convert m->km
            if m == "betweenness":
                tmp, = ax.plot([0,1], [analysis_mst_result_AVG[key]/1000, analysis_mst_result_AVG[key]/1000], **plotparam_analysis["mst"])
            ax.plot(prune_quantiles, analysis_result_AVG[m][key]/1000, **plotparam_analysis["bikegrown_" + m])
        else:
            if m == "betweenness":
                tmp, = ax.plot([0,1], [analysis_mst_result_AVG[key], analysis_mst_result_AVG[key]], **plotparam_analysis["mst"])
            ax.plot(prune_quantiles, analysis_result_AVG[m][key], **plotparam_analysis["bikegrown_" + m])
        
        if key == "efficiency_local" and plotconstricted and m == "betweenness": # Only show betweenness - the others are similar
            tmp, = ax.plot(prune_quantiles, analysis_constricted_AVG[m][1], **plotparam_analysis["constricted"])
            tmp.set_label('_hidden')
                
        if i == 0:
            ax.set_ylim(0, ymax0)
        set_analysissubplot(key)
        set_analysissubplot_special(key, poi_source)
        ax.set_title(list(keys_metrics.values())[i+nc])
        ax.set_xlabel('Growth strategy quantile')
        if key in ["poi_coverage"]:
            # https://stackoverflow.com/questions/30914462/matplotlib-how-to-force-integer-tick-labels
            ax.set_yticks([0,40,80,120,160])
        

if plotnotitle:
    plt.subplots_adjust(top = 0.92, bottom = 0.14, left = 0.05, right = 0.97, wspace = 0.28, hspace = 0.28)
else:
    plt.subplots_adjust(top = 0.82, bottom = 0.14, left = 0.05, right = 0.97, wspace = 0.28, hspace = 0.28)
if plotconstricted:
    fig.savefig(PATH["plots"] + "/" + 'averagecity_analysis_poi_' + poi_source + '.eps', facecolor = "white", edgecolor = 'none')
else:
    fig.savefig(PATH["plots"] + "/" + 'averagecity_analysis_poi_' + poi_source + '_noconstr.eps', facecolor = "white", edgecolor = 'none')
plt.close()

### Directness comparisons

In [ ]:
# %%capture
plotparam_analysis["constricted"] = {"linewidth": 1.5, "color": '#D22A0E', "linestyle": "solid", "label": "Betw. cars"}
plotparam_analysis["mst"] = {"linewidth": 1, "color": '#aaaaaa', "linestyle": "dotted", "label": "MST"}
plotparam_analysis["bikegrown_betweenness"] = {"linewidth": 2.5, "color": '#0eb6d2', "linestyle": "solid", "label": "Betw."}
plt.style.use(PATH["parameters"] + 'plotstyle.mplstyle')
# Run all parameter sets
prune_measure_list = ["betweenness", "closeness", "random"]
poi_source = "grid" # railwaystation grid

analysis_result_city = {}
analysis_result_AVG = {}
analysis_constricted_AVG = {}
analysis_mst_result_city = {}
for m in prune_measure_list:
    analysis_result_city[m] = {}
    numcities = 0
    for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):

        # PLOT Analysis
        filename = placeid + '_poi_' + poi_source + "_mst.csv"
        analysis_mst_result_city_temp = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
        
        filename = placeid + '_poi_' + poi_source + "_" + m + ".csv"
        analysis_result_city_temp = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
        if len(analysis_result_city_temp) == 0: # Discard if no results (for example no railwaystations)
            print(placeid + ": No analysis results available")
            continue
        else:
            numcities += 1
            analysis_result_city[m][placeid] = analysis_result_city_temp
            metric_keys = analysis_result_city[m][placeid].dtype.names
            analysis_mst_result_city[placeid] = analysis_mst_result_city_temp

    # All cities are loaded, now create the average values
    analysis_result_AVG[m] = {}
    for metric in metric_keys:
        temp = np.zeros([40, numcities])
        for i, placeid_this in enumerate(analysis_result_city[m].keys()):
            temp[:, i] = analysis_result_city[m][placeid_this][metric]
        analysis_result_AVG[m][metric] = np.mean(temp, axis = 1)
        
    analysis_mst_result_AVG = {}
    for metric in metric_keys:
        temp = np.zeros([1, numcities])
        for i, placeid_this in enumerate(analysis_mst_result_city.keys()):
            temp[:, i] = analysis_mst_result_city[placeid_this][metric]
        analysis_mst_result_AVG[metric] = np.mean(temp, axis = 1)
    print(analysis_mst_result_AVG)

    analysis_constricted_AVG[m] = {}
    for metric in list(range(9)): # 9 constricted fields are loaded
        temp = np.zeros([40, len(analysis_constricted[poi_source][m].keys())]) 
        for i, placeid_this in enumerate(analysis_constricted[poi_source][m].keys()):
            temp[:, i] = analysis_constricted[poi_source][m][placeid_this][:, metric]
        analysis_constricted_AVG[m][metric] = np.mean(temp, axis = 1)

# Plot
nc = 4
fig, axes = plt.subplots(nrows = 1, ncols = nc, figsize=(640/plotparam["dpi"], 180/plotparam["dpi"]), dpi=plotparam["dpi"])
keys_metrics = {"directness_all_linkwise": "All CC, avg. of ratios", "directness": "All CC, ratio of totals", "directness_lcc_linkwise": "LCC, avg. of ratios", "directness_lcc": "LCC, ratio of totals", }

for i, ax in enumerate(axes):
    for m in prune_measure_list:
        key = list(keys_metrics.keys())[i]
        ax.plot(prune_quantiles, analysis_result_AVG[m][key], **plotparam_analysis["bikegrown_" + m])
    tmp, = ax.plot([0,1], [analysis_mst_result_AVG[key], analysis_mst_result_AVG[key]], **plotparam_analysis["mst"])
    ax.set_ylim(bottom = 0.43)
    ax.set_ylim(top = 0.84)
    ax.set_title(list(keys_metrics.values())[i])
    ax.set_xlabel('Growth strategy quantile')
    ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_xlim(0, 1)
    if i == 0:
        ax.set_ylabel('Directness')
#     if i == 3:
#         leg = ax.legend(loc = 'best')
#         leg.get_frame().set_linewidth(0.5)
#         ax.set_zorder(100)

if plotnotitle:
    plt.subplots_adjust(top = 0.86, bottom = 0.2, left = 0.063, right = 0.98, wspace = 0.28, hspace = 0.28)
else:
    plt.subplots_adjust(top = 0.86, bottom = 0.2, left = 0.063, right = 0.98, wspace = 0.28, hspace = 0.28)
if plotconstricted:
    fig.savefig(PATH["plots"] + "/" + 'averagecity_directnesscomparisons_poi_' + poi_source + '.eps', facecolor = "white", edgecolor = 'none')
else:
    fig.savefig(PATH["plots"] + "/" + 'averagecity_directnesscomparisons_poi_' + poi_source + '_noconstr.eps', facecolor = "white", edgecolor = 'none')
plt.close()

### Overlaps with existing bike infra

In [ ]:
from matplotlib.ticker import PercentFormatter

# %%capture
plt.style.use(PATH["parameters"] + 'plotstyle.mplstyle')
# Run all parameter sets
prune_measure_list = ["betweenness", "closeness", "random"]
poi_source = "railwaystation" # railwaystation grid

analysis_result_city = {}
analysis_result_AVG = {}
for m in prune_measure_list:
    analysis_result_city[m] = {}
    numcities = 0
    for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):

        # PLOT Analysis
        filename = placeid + '_poi_' + poi_source + "_" + m + ".csv"
        analysis_result_city_temp = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
        if len(analysis_result_city_temp) == 0: # Discard if no results (for example no railwaystations)
            print(placeid + ": No analysis results available")
            continue
        else:
            numcities += 1
            analysis_result_city[m][placeid] = analysis_result_city_temp
            metric_keys = analysis_result_city[m][placeid].dtype.names

    # All cities are loaded, now create the average values
    analysis_result_AVG[m] = {}
    for metric in metric_keys:
        temp = np.zeros([40, numcities])
        for i, placeid_this in enumerate(analysis_result_city[m].keys()):
            temp[:, i] = analysis_result_city[m][placeid_this][metric]
        analysis_result_AVG[m][metric] = np.mean(temp, axis = 1)

# Plot
nc = 1
fig, axes = plt.subplots(nrows = 2, ncols = nc, figsize=(140/plotparam["dpi"], 320/plotparam["dpi"]), dpi=plotparam["dpi"])
keys_metrics = {"overlap_biketrack": "Overlap Protected", "overlap_bikeable": "Overlap Bikeable"}

i = 0
ax = axes[i]
key = list(keys_metrics.keys())[i]
for m in prune_measure_list:
    ax.plot(prune_quantiles, analysis_result_AVG[m][key] / analysis_result_AVG[m]["length"], **plotparam_analysis["bikegrown_" + m])
    if not plotnotitle:
        ax.text(-0.15, ymax0*1.25, "Average city" + " (" + poi_source + ")", fontsize=16, horizontalalignment='left')

set_analysissubplot(key)
# set_analysissubplot_special(key, poi_source)
ax.set_title(list(keys_metrics.values())[i])
ax.set_xlabel('')
ax.set_xticklabels([])
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))

# Inset Copenhagen
ax = fig.add_axes([0.54, 0.71, 0.39, 0.20]) # inset axes
cityexample = "copenhagen"
for m in prune_measure_list:
    ax.plot(prune_quantiles, analysis_result[p][m][cityexample][key] / analysis_result[p][m][cityexample]["length"], **plotparam_analysis["bikegrown_" + m])
    ax.set_ylim(0.42,0.88)
    ax.set_yticks([0.5,0.6,0.7,0.8])
    ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
    ax.text(0.96, 0.8, "Copenhagen", horizontalalignment = "right", fontsize = 6)
set_analysissubplot(key)
ax.set_xticklabels([])



i = 1
ax = axes[i]
key = list(keys_metrics.keys())[i]
for m in prune_measure_list:
    ax.plot(prune_quantiles, analysis_result_AVG[m][key] / analysis_result_AVG[m]["length"], **plotparam_analysis["bikegrown_" + m])
#     leg = ax.legend(loc = 'best')
#     leg.get_frame().set_linewidth(0.5)
    
set_analysissubplot(key)
# set_analysissubplot_special(key, poi_source)
ax.set_title(list(keys_metrics.values())[i])
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
ax.set_xlabel('Growth strategy quantile')

if plotnotitle:
    plt.subplots_adjust(top = 0.92, bottom = 0.11, left = 0.21, right = 0.95, wspace = 0.28, hspace = 0.28)
else:
    plt.subplots_adjust(top = 0.82, bottom = 0.11, left = 0.21, right = 0.95, wspace = 0.28, hspace = 0.28)

fig.savefig(PATH["plots"] + "/" + 'averagecity_analysisoverlap_poi_' + poi_source + '.eps', facecolor = "white", edgecolor = 'none')
plt.close()

### Constricted analysis plots

In [ ]:
def set_constrictedsubplot_special(key):
    if key == 0:
        ax.set_ylim(0.6,0.8)
    elif key == 1:
        ax.set_ylim(0.095,0.115)
    elif key == 2:
        ax.set_ylim(0.6,0.82)
    elif key == 3:
        ax.set_ylim(0.58,0.8)
    elif key == 4:
        ax.set_ylim(0.48,0.57)   

In [ ]:
# %%capture

fig, axess = plt.subplots(nrows = 6, ncols = 5, figsize=(640/plotparam["dpi"], 6*140/plotparam["dpi"]), dpi=plotparam["dpi"])


analysis_constricted_AVG = {}
axescount = 0
for p in ["grid","railwaystation"]:
    analysis_constricted_AVG[p] = {}
    for m in prune_measure_list:
        analysis_constricted_AVG[p][m] = {}
        for metric in list(range(9)): # 9 constricted fields are loaded
            temp = np.zeros([40, len(analysis_constricted[p][m].keys())]) 
            for i, placeid_this in enumerate(analysis_constricted[p][m].keys()):
                temp[:, i] = analysis_constricted[p][m][placeid_this][:, metric]
            analysis_constricted_AVG[p][m][metric] = np.mean(temp, axis = 1)

        axes = axess[axescount]
        axescount += 1
        for i, ax in enumerate(axes):
            if axescount == 1:
                if i != 2:
                    ax.set_title(constricted_plotinfo["title"][i])
                else:
                    ax.set_title("Directness")
            if axescount == 6:
                ax.set_xlabel('Growth strategy quantile')
            ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
            if axescount != 6:
                ax.set_xticklabels([])
            if i == 0:
                ax.set_ylabel(p + "\n" + m)
            ax.set_xlim(-0.05, 1.05)
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            set_constrictedsubplot_special(i)

            # Efficiency global
            if i == 0:
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][0], **plotparam_analysis["constricted_SI"])
                ymin, ymax = ax.get_ylim()

            # Efficiency local
            elif i == 1:
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][1], **plotparam_analysis["constricted_SI"])

            # Directness
            elif i == 2:
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][8], **plotparam_analysis["constricted_SI"])

            # Clustering
            elif i == 3:
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][2], **plotparam_analysis["constricted_10"])
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][3], **plotparam_analysis["constricted_5"])
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][4], **plotparam_analysis["constricted_3"])
#                 ax.legend(loc = 'best')
                if axescount == 1:
                    ax.text(0.45, 0.735, plotparam_analysis["constricted_10"]["label"], fontsize = 7, color = plotparam_analysis["constricted_10"]["color"])
                    ax.text(0.45, 0.655, plotparam_analysis["constricted_5"]["label"], fontsize = 7, color = plotparam_analysis["constricted_5"]["color"])
                    ax.text(0.45, 0.6, plotparam_analysis["constricted_3"]["label"], fontsize = 7, color = plotparam_analysis["constricted_3"]["color"])

            # Anisotropy
            elif i == 4:
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][5], **plotparam_analysis["constricted_10"])
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][6], **plotparam_analysis["constricted_5"])
                ax.plot(prune_quantiles, analysis_constricted_AVG[p][m][7], **plotparam_analysis["constricted_3"])

plt.subplots_adjust(top = 0.95, bottom = 0.05, left = 0.09, right = 0.97, wspace = 0.35)
fig.savefig(PATH["plots"] + "/" + 'averagecity_constricted.eps', facecolor = "white", edgecolor = 'none')

## Basic stats on all cities

### World map

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[(world.name!="Antarctica")] #### exclude Antartica


cities = pd.read_csv(PATH['parameters'] + 'cities_all_lat_lng.csv') #### read cities lat and lon

cities_x = np.array(cities['lng'].tolist())
cities_y = np.array(cities['lat'].tolist())


fig, ax = plt.subplots(figsize=(640/plotparam["dpi"], 320/plotparam["dpi"]), dpi=plotparam["dpi"], squeeze = True)
ax.set_facecolor('aliceblue')  #### color background with blue to represent oceans


world.plot(ax=ax,color='lightgrey', edgecolor='whitesmoke', linewidth=0.5)
plt.plot(cities_x,cities_y,'.',color='black', markersize=6) # 'o',markersize=5, markerfacecolor='black',markeredgewidth=0.8, markeredgecolor='grey'
ax.set_ylim([-65, 90])
ax.set_xlim([-180, 180])
plt.xticks([])
plt.yticks([])



###################  Europe inset
axins = ax.inset_axes([0.01, 0.01, 0.251, 0.48]) ### location and size of the inset [x_0,y_0, width, height]

world.plot(color='lightgrey', edgecolor='whitesmoke',ax=axins, linewidth=0.6)
axins.plot(cities_x,cities_y,'.',color='black', markersize=7)
axins.set_facecolor('aliceblue')
# sub region of the original image
x1, x2, y1, y2 = -5, 22, 40.5, 61 #### coordinates to zoom in
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticks([])
axins.set_yticks([])
# ax.indicate_inset_zoom(axins, edgecolor="black")  ### lines towards the inset
plt.tight_layout()
# plt.subplots_adjust(top = 1.12, bottom = 0, left = 0, right = 1)
fig.savefig('world_insets.eps',bbox='tight')

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[(world.name!="Antarctica")] #### exclude Antartica


cities = pd.read_csv(PATH['parameters'] + 'cities_all_lat_lng.csv') #### read cities lat and lon

cities_x = np.array(cities['lng'].tolist())
cities_y = np.array(cities['lat'].tolist())


fig = plt.figure(figsize=(310/plotparam["dpi"], 240/plotparam["dpi"]), dpi=plotparam["dpi"])

ax = fig.add_subplot(3,2,(1,4))
ax.set_facecolor('aliceblue')  #### color background with blue to represent oceans


world.plot(ax=ax,color='lightgrey', edgecolor='whitesmoke', linewidth=0.4)
plt.plot(cities_x,cities_y,'.',color='black', markersize=1.5) # 'o',markersize=5, markerfacecolor='black',markeredgewidth=0.8, markeredgecolor='grey'
ax.set_ylim([-65, 90])
ax.set_xlim([-180, 180])
plt.xticks([])
plt.yticks([])


###################  Europe inset
axins = ax.inset_axes([0.01, 0.01, 0.251, 0.48]) ### location and size of the inset [x_0,y_0, width, height]

world.plot(color='lightgrey', edgecolor='whitesmoke',ax=axins, linewidth=0.5)
axins.plot(cities_x,cities_y,'.',color='black', markersize=2)
axins.set_facecolor('aliceblue')
# sub region of the original image
x1, x2, y1, y2 = -5, 22, 40.5, 61 #### coordinates to zoom in
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticks([])
axins.set_yticks([])
# ax.indicate_inset_zoom(axins, edgecolor="black")  ### lines towards the inset
plt.tight_layout()
# plt.subplots_adjust(top = 1.12, bottom = 0, left = 0, right = 1)




# Left subplot: Lengths
axes = fig.add_subplot(325)
axes.hist([i/1000 for i in datastats["length_biketrack"]], bins = np.linspace(0,1400,15), edgecolor="w")
axes.set_ylim([0, 19])
axes.set_yticks([0,5,10,15]);
axes.set_xlabel("Bicycle network length [km]")
axes.set_ylabel("Cities")
axes.set_xticks([0,400,800,1200]);
labels = axes.set_xticklabels([0,400,800,1200])
# https://stackoverflow.com/questions/14852821/aligning-rotated-xticklabels-with-their-respective-xticks
for i, label in enumerate(labels):
    labelypos = label.get_position()[1]

# Right subplot: Components
axes = fig.add_subplot(326)
axes.hist([i for i in datastats["components"] if i != 0], bins=np.logspace(0, 4, 17), edgecolor="w")

axes.set_ylim([0, 19])
axes.set_yticks([0,5,10,15]);
axes.set_yticklabels([])
axes.set_xlabel("Disconnected components");
# labels = axes.set_xticklabels([1,10,100,1000,10000])
axes.set_xscale('log') 
axes.set_xticks([1,10,100,1000,10000]);
# for i, label in enumerate(labels):
#     label.set_y(labelypos)

# # Right subplot: Street Lengths
# axes = fig.add_subplot(326)
# axes.hist([i/1000 for i in datastats["length_carall"]], bins = np.linspace(0,14000,15), edgecolor="w", color="red")
# axes.set_ylim([0, 19])
# axes.set_yticks([0,5,10,15]);
# axes.set_yticklabels([])
# axes.set_xlabel("Street network length [km]")
# axes.set_xticks([0,4000,8000,12000]);
# labels = axes.set_xticklabels([0,4000,8000,12000])

    

plt.subplots_adjust(top = 1, bottom = 0.16, left = 0.12, right = 0.99, wspace = 0.1, hspace = 0.1)
fig.savefig(PATH["plots"] + 'world_insets.eps',bbox='tight')

### Stats / Histograms

In [ ]:
datastats = {}
datastats["length_biketrack"] = []
datastats["length_biketrack_onstreet"] = []
datastats["length_carall"] = []
datastats["components"] = []
for placeid, placeinfo in cities.items():
    datastats["length_biketrack"].append(analysis_existing[placeid]["length"][analysis_existing_rowkeys["biketrack"]])
    datastats["length_biketrack_onstreet"].append(analysis_existing[placeid]["length"][analysis_existing_rowkeys["biketrack_onstreet"]])
    datastats["length_carall"].append(analysis_existing[placeid]["length"][analysis_existing_rowkeys["carall"]])
    datastats["components"].append(analysis_existing[placeid]["components"][analysis_existing_rowkeys["biketrack"]])

In [ ]:
datastats

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(320/plotparam["dpi"], 120/plotparam["dpi"]), dpi=plotparam["dpi"], squeeze = True)

# Left subplot: Lengths
axes = ax[0]
axes.hist([i/1000 for i in datastats["length_biketrack"]], bins = np.linspace(0,1400,15), edgecolor="w")
axes.set_ylim([0, 19])
axes.set_xlabel("Bicycle network length [km]")
axes.set_ylabel("Number of cities")
axes.set_xticks([0,400,800,1200]);

# Right subplot: Components
axes = ax[1]
axes.hist([i for i in datastats["components"] if i != 0], bins=np.logspace(0, 4, 17), edgecolor="w")
axes.set_xscale('log') 
axes.set_ylim([0, 19])
axes.set_yticklabels([])
axes.set_xlabel("Disconnected components");
axes.set_xticks([1,10,100,1000,10000]);


In [ ]:
plt.hist(datastats["length_biketrack"])

In [ ]:
# plt.hist(np.log10([i for i in datastats["components"] if i != 0]), bins = 20)

fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.hist([i for i in datastats["components"] if i != 0], bins=np.logspace(0, 4, 20), edgecolor="w")

axes.set_xscale('log') 


In [ ]:
plt.hist([i/j for (i,j) in zip(datastats["length_biketrack"], datastats["length_carall"])], bins = 50)

In [ ]:
[(i/j, p) for (i,j,p) in zip(datastats["length_biketrack"], datastats["length_carall"], list(cities.keys()))]

In [ ]:
plt.hist([i/j for (i,j) in zip(datastats["length_biketrack_onstreet"], datastats["length_carall"])])